In [48]:
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
# nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

import re
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

import gensim
from gensim import corpora
from gensim.summarization.summarizer import summarize
import pickle


In [3]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

### Load training data

In [4]:
# load file
training_feedback = pd.read_csv('training_data/feedback_data_for_training.csv')

In [5]:
training_feedback

,source,feedback_text,topic,sub_topic
0,team_generated,i have no idea how i’m doing so far. we’ve tur...,feedback,slow turnaround
1,team_generated,"i’d ask for more time from the TAs, or try to ...",feedback,slow turnaround
2,team_generated,the way that they encourage lots of class disc...,live session,good discussion
3,team_generated,"async is going pretty well, but not super appl...",async,disconnected from live session
4,team_generated,abe lincoln is the best professor i’ve ever had,instructor,NaN
5,team_generated,tech is bad,tech,negative
6,team_generated,this is a great bot,tech,positive
7,team_generated,"pretty well, actually! i think i’m finally get...",course material,understandable
8,team_generated,"the breakout sessions were very engaging, and ...",live session,good discussion
9,team_generated,one person dominated the full-class discussion...,live session,unbalanced


In [9]:
topics = set(training_feedback.topic)
topics

{'assignments',
 'async',
 'course material',
 'feedback',
 'instructor',
 'instructors',
 'live session',
 'misc',
 nan,
 'office hours',
 'tech'}

### Create subtopic models, by topic

In [37]:
for topic in topics:
    print ("Processing", topic)
    topic_subset = training_feedback[training_feedback.topic==topic]
    topic_text = topic_subset.feedback_text
    # print(text)
    
    chats_tokenized = []
    
    for t in topic_text:
        tokens = prepare_text_for_lda(t)
        chats_tokenized.append(tokens)
        
    dictionary = corpora.Dictionary(chats_tokenized)
    dictionary_filename = "topic_dictionaries/%s_dictionary_feedback.gensim" % (str(topic))
    corpus = [dictionary.doc2bow(text) for text in chats_tokenized]
    corpus_filename = "topic_corpi/%s_corpus_feedback.pkl" % (str(topic))
    

    pickle.dump(corpus, open(corpus_filename, 'wb'))
    dictionary.save(dictionary_filename)

Processing nan
Processing misc
Processing instructors
Processing feedback
Processing tech
Processing async
Processing assignments
Processing instructor
Processing office hours
Processing live session
Processing course material


In [44]:
NUM_TOPICS = 3
NUM_PASSES = 15

for topic in topics:
    print(topic)
    dictionary_filename = "topic_dictionaries/%s_dictionary_feedback.gensim" % (str(topic))
    dictionary = corpora.Dictionary.load(dictionary_filename)
    corpus_filename = "topic_corpi/%s_corpus_feedback.pkl" % (str(topic))
    with open(corpus_filename, 'rb') as f:
        corpus = pickle.load(f)
    if (len(dictionary) == 0):
        continue
    
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=NUM_PASSES)
    ldamodel_filename = 'topic_models/feedback_model%st_%sp.gensim' % (str(NUM_TOPICS), str(NUM_PASSES))
    ldamodel.save(ldamodel_filename)
    
    sub_topics = ldamodel.print_topics(num_words=5)
    for sub_topic in sub_topics:
        print(sub_topic)
    
    


nan
misc
(0, '0.045*"upcoming" + 0.045*"appear" + 0.045*"understaffed" + 0.045*"badly" + 0.045*"program"')
(1, '0.049*"reading" + 0.028*"right" + 0.028*"grade" + 0.028*"light" + 0.028*"pretty"')
(2, '0.029*"system" + 0.029*"speaker" + 0.029*"still" + 0.029*"material" + 0.029*"assignment"')
instructors
(0, '0.042*"class" + 0.030*"background" + 0.030*"engage" + 0.017*"instructor" + 0.017*"great"')
(1, '0.032*"class" + 0.032*"really" + 0.022*"material" + 0.022*"intuition" + 0.022*"focus"')
(2, '0.029*"responsive" + 0.029*"people" + 0.029*"sessions" + 0.028*"instructor" + 0.017*"prefer"')
feedback
(0, '0.058*"grade" + 0.042*"feedback" + 0.037*"assignment" + 0.032*"instructor" + 0.022*"though"')
(1, '0.057*"feedback" + 0.026*"grade" + 0.021*"get" + 0.021*"assignment" + 0.020*"might"')
(2, '0.065*"feedback" + 0.034*"semester" + 0.026*"receive" + 0.026*"presentation" + 0.018*"frustrate"')
tech
(0, '0.046*"video" + 0.046*"teacher" + 0.046*"watch" + 0.026*"mobile" + 0.026*"async"')
(1, '0.046*"

### Generate topic summaries

In [63]:
for topic in topics:
    print ("Processing", topic)
    topic_subset = training_feedback[training_feedback.topic==topic]
    topic_text = topic_subset.feedback_text
    
    s = '. '
    text = s.join(topic_text)
    text = re.sub("\.+", ".", text)
    text = re.sub("\n", "", text)
    # print(text)
    print(summarize(text))
    print('\n')

Processing nan



Processing misc
The week off for fall immersion seemed unexpected to both of my professors when it came to planning class material/projects/assignments.
We were still expected to do assignments, readings, and async material.


Processing instructors
They don’t seem to have a ton of extra time to be developing the materials or a love of pedagogy like the W261 team, so the class times are more along the lines of re-hashing the async and catching up on recent articles in the AI space via student-presented “Cloud News.
He also treated the class like grownups and didn't try to police folks working together.
I liked that he'd bring additional examples (from his work and elsewhere) to class to complement the official material, too.
really good, designed the whole course, hold office hours, explain concepts in great clarity & really responsive on piazza & mail.
He explains things well, and has a really good approach to working through proofs "live" in class on an iPad. very e